# Setup

In [180]:
import sys
import importlib

src_path = '/Users/jblandin/my/school-local/cs511/pywumpuslite'

def reload_module_by_name(mod_name, var_name, path):
    if path is not None and path not in sys.path:
        sys.path.append(path)
        print('Appended {} to sys.path'.format(path))
    else:
        print('{} already in sys.path'.format(path))
                
    for mod in list(sys.modules.keys()):
        if mod_name in mod:
            del sys.modules[mod]
    
    if var_name in globals():
        del globals()[var_name] # deletes the variable named <var_name>
    return importlib.__import__(mod_name)

def reload(path=src_path):
    global Action, agent_function, agent, environment, simulation, transfer_percept, world_application
    agent_function = reload_module_by_name('agent_function', 'agent_function', path=path)
    agent = reload_module_by_name('agent', 'agent', path=path)
    environment = reload_module_by_name('environment', 'environment', path=path)
    simulation = reload_module_by_name('simulation', 'simulation', path=path)
    transfer_percept = reload_module_by_name('transfer_percept', 'transfer_percept', path=path)
    world_application = reload_module_by_name('world_application', 'world_application', path=path)
    Action = reload_module_by_name('action', 'Action', path=path)
    import agent_function, agent, environment, simulation, transfer_percept, world_application
    
reload(src_path)

/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path


# Run

In [280]:
reload()
print()
out_writer = open('wumpus_output.txt', 'w')

world_application.run_application(num_trials=1, max_steps=50)

/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path
/Users/jblandin/my/school-local/cs511/pywumpuslite already in sys.path

Dimensions: 4 x 4
Maximum number of steps: 50
Number of trials: 1
Random agent location: False
Random number seed: 165355
Output filename: wumpus_output.txt
Non-Deterministic Behavior: False

 -----------------------
|     |     |     |     | 
| G   |     |     |     | 
 -----------------------
|     |     |     |     | 
|     |     |     |     | 
 -----------------------
| P W |     |     |     | 
|     |     |     |     | 
 -----------------------
|     |     | P   |     | 
|   > |     |     |  

 ----  ----  ----  ---- 

 ----  ----  ----  ---- 

 ----  ----  ----  ---- 

 ----  ----  ----  ---- 


Explored
--------

 ----  ----  ----  ---- 

 ----  ----  ----  ---- 

 ----  ----  ----  ---- 

 1.00  ----  ----  ---- 


Posterior Death probs
---------------------

 0.07  0.07  0.07  0.07 

 0.07  0.07  0.07  0.07 

 1.00  0.07  0.07  0.07 

 ----  0.54  0.07  0.07 


---------------------------------------------------------------------------


 -----------------------
|     |     |     |     | 
| G   |     |     |     | 
 -----------------------
|     |     |     |     | 
|     |     |     |     | 
 -----------------------
| P W |     |     |     | 
|     |     |     |     | 
 -----------------------
|     |     | P   |     | 
|   > |     |     |     | 
 -----------------------


Percept: < none, none, Breeze, Stench, none >

Current score: -10
Last action: NO_OP
Time step: 41
planned_actions: [6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,

# Scratch

In [181]:
def _adjacent_locs(loc, world_size=4):
    """Returns all adjacent locations."""
    adj = []
    x, y = loc
    # square below
    if y > 0:
        adj.append([x, y-1])
    # square above
    if y < world_size-1:
        adj.append([x, y+1])
    # square to the left
    if x > 0:
        adj.append([x-1, y])
    # square to the right
    if x < world_size-1:
        adj.append([x+1, y])
    return adj


def _forward_loc(agent_loc, agent_dir, world_size=4):
    """Returns location in front of agent. If facing a wall, return None."""
    x, y = agent_loc
    if agent_dir == 'N':
        if y < world_size-1:
            return [x, y+1]
        else:
            return None
    elif agent_dir == 'E':
        if x < world_size-1:
            return [x+1, y]
        else:
            return None
    elif agent_dir == 'S':
        if y > 0:
            return [x, y-1]
        else:
            return None
    elif agent_dir == 'W':
        if x > 0:
            return [x-1, y]
        else:
            return None

def _turn_left_dir(agent_dir):
    """Returns new direction after turning left."""
    if agent_dir == 'N':
        new_agent_dir = 'W'
    elif agent_dir == 'E':
        new_agent_dir = 'N'
    elif agent_dir == 'S':
        new_agent_dir = 'E'
    elif agent_dir == 'W':
        new_agent_dir = 'S'
    return new_agent_dir


def _turn_right_dir(agent_dir):
    """Returns new direction after turning right."""
    if agent_dir == 'N':
        new_agent_dir = 'E'
    elif agent_dir == 'E':
        new_agent_dir = 'S'
    elif agent_dir == 'S':
        new_agent_dir = 'W'
    elif agent_dir == 'W':
        new_agent_dir = 'N'
    return new_agent_dir



In [262]:
def _path_to_loc(cur_loc, dest_loc, explored_locs):
    """
    Using Depth First Search, computes the path from the current loc to the destination loc.
    Only explored nodes will be used to get there.
    The object returned will be the path of the Node, which is a sequence of locs.
    """
    
    class Node:
        def __init__(self, parent_node, loc):
            self.parent_node = parent_node
            self.loc = loc
        def path(self):
            """Builds the path of nodes obtained to get here."""
            path = [self.loc]
            if self.parent_node is None:
                return path
            else:
                return self.parent_node.path() + path
    
    cur_loc_node = Node(None, cur_loc)
    
    e = []  # explored (note that this is different than explored_locs param)
    f = []  # frontier
    
    # Initial goal check
    if dest_loc == cur_loc:
        return cur_loc_node.path()
    
    e.append(cur_loc)
    
    # Build initial frontier with locs adj. to cur. loc
    adj_to_cur = _adjacent_locs(cur_loc)
    for l in (explored_locs + [dest_loc]):
        if l in adj_to_cur:
            f.insert(0, Node(cur_loc_node, l))
    
    def _expand(parent):
        adj_locs = _adjacent_locs(parent.loc)
        adj_and_explored_locs = []
        for l in adj_locs:
            if (l in explored_locs or l == dest_loc) and l not in e:
                child = Node(parent, l)
                adj_and_explored_locs.append(child)
        return adj_and_explored_locs
        
    while len(f) > 0:
        n = f.pop()
        e.append(n.loc)
        if n.loc == dest_loc:
            return n.path()
        else:
            f = _expand(n) + f
                
    raise Exception('No action sequence found')
        

def _action_seq_to_adj_loc(cur_loc, cur_dir, dest_loc, actions=None):
    """Called recursively.
    Given current loc, current direction, and the destination loc, which must be adjacent to current loc,
    computes the action sequence needed to get there as well as the final direction.
    """
    if actions is None:
        actions = []
    # Check stopping criteria
    if _forward_loc(cur_loc, cur_dir) == dest_loc:
        actions.append(Action.GO_FORWARD)
        return actions, cur_dir
    else:
        # Turn Left (could also turn Right)
        # TODO 03/11/2019 
        actions.append(Action.TURN_LEFT)
        new_dir = _turn_left_dir(cur_dir)
        # Recurse
        return _action_seq_to_adj_loc(cur_loc, new_dir, dest_loc, actions)
    
         
def _action_seq_to_non_adj_loc(cur_loc, cur_dir, dest_loc, explored_locs):
    """
    Given current loc, current direction, and the destination loc, computes the action sequence needed to get there.
    Uses depth first search by calling _path_to_loc.
    """
    path = _path_to_loc(cur_loc, dest_loc, explored_locs)
    d = cur_dir
    actions = []
    for idx in range(len(path)-1):
        actions, d = _action_seq_to_adj_loc(path[idx], d, path[idx+1], actions)
    return actions
    

In [263]:
_path_to_loc(cur_loc=[0,0],
             dest_loc=[0,1],
             explored_locs=[[0,0]])

[[0, 0], [0, 1]]

In [264]:
_action_seq_to_adj_loc([2,0], 'S', [1,0])

([3, 3, 3, 1], 'W')

In [265]:
seq =_action_seq_to_non_adj_loc(cur_loc=[2,0],
                           cur_dir='E',
                           dest_loc=[0,2],
                           explored_locs=[[0,0], [1,0], [2,0], [0,1]])

Action.print_actions(seq)

['TURN_LEFT',
 'TURN_LEFT',
 'GO_FORWARD',
 'GO_FORWARD',
 'TURN_LEFT',
 'TURN_LEFT',
 'TURN_LEFT',
 'GO_FORWARD',
 'GO_FORWARD']